In [16]:
import os

# Utilities
import Utils.folder_utils as folder_utils
import Utils.TIMIT_utils as TIMIT_utils
import Utils.image_utils as image_utils

# Config
import Utils.Eva_config_consts as config

# Sound utils
from Utils.nist_reader import NistReader
import Utils.sound_utils as sound_utils

# Spectrograms
from Utils.SpectrogramFactory import SpectrogramFactory
from Utils.Spectrogram import Spectrogram

import Utils.Spectrogram as SpectrogramUtils

%matplotlib inline

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
#sound
path_to_flac = "Data/TIMIT/timit/train"
path_to_flac_test_folder = "TestFolder/TestData/fcjf0"
path_to_flac_test_folder_track = "TestFolder/TestData/fcjf0/SA1.WAV"

#temp
temp_folder = "TestFolder/Temp"
temp_phonemes_folder = "TestFolder/TempPhonemes"
test_output = "TestFolder/Temp/results.wav"
#temp_image_to_process = "/Volumes/Storage/processing.jpg"

In [18]:
pathes = folder_utils.reverse_folder(path_to_flac_test_folder, ".WAV")
pathes

['TestFolder/TestData/fcjf0/SA1.WAV',
 'TestFolder/TestData/fcjf0/SA2.WAV',
 'TestFolder/TestData/fcjf0/SI1027.WAV',
 'TestFolder/TestData/fcjf0/SI1657.WAV',
 'TestFolder/TestData/fcjf0/SI648.WAV',
 'TestFolder/TestData/fcjf0/SX127.WAV',
 'TestFolder/TestData/fcjf0/SX217.WAV',
 'TestFolder/TestData/fcjf0/SX307.WAV',
 'TestFolder/TestData/fcjf0/SX37.WAV',
 'TestFolder/TestData/fcjf0/SX397.WAV']

In [19]:
print "Window size: " + str(config.WINDOW_SIZE)
print "Window step: " + str(config.WINDOW_STEP)
nistReader = NistReader()
spectrogramFactory = SpectrogramFactory(window_size=config.WINDOW_SIZE, window_step=config.WINDOW_STEP)

Window size: 256
Window step: 128


In [20]:
path = pathes[0]
#sentence
print path
phonemes = TIMIT_utils.parse_phoneme_file(path)
speaker = folder_utils.get_speaker_name(path)

temp_speaker_folder = os.path.join(temp_folder, speaker)
if not os.path.exists(temp_speaker_folder):
    os.makedirs(temp_speaker_folder)
    
#converting to wave
wav_file = nistReader.Nist2Wav(path, temp_speaker_folder)

for i in range(len(phonemes)):
    phoneme = phonemes[i]
    #Cutting one phoneme
    if i == 0 or i == len(phonemes):
        start = int(phoneme[0])
        end = int(phoneme[1])
    else:
        start = int(phoneme[0]) - config.PHONEME_OFFSET
        end = int(phoneme[1]) + config.PHONEME_OFFSET
        
    # TODO: create a phoneme object
    phone_file = sound_utils.cutPhonemeChunk(wav_file, temp_phonemes_folder, start, end, phoneme[2])
    spectrogram = spectrogramFactory.create_spectrogram(phone_file)

TestFolder/TestData/fcjf0/SA1.WAV


# Testing

In [21]:
spectrogramObj = spectrogramFactory.create_spectrogram(wav_file)
spectrogram = spectrogramObj.spectrogram_values

waveform = SpectrogramUtils.InvertSpectrogram(spectrogram=spectrogram, window_size=config.WINDOW_SIZE,
                                              window_step=config.WINDOW_STEP)
sound_utils.save_to_wav(test_output, waveform)

In [22]:
print spectrogram.shape
print "input: " + wav_file
print "result: " + test_output

(512, 364)
input: TestFolder/Temp/fcjf0/SA1.wav
result: TestFolder/Temp/results.wav


## Experiments

In [33]:
from Utils.sound_utils import get_wav_info

In [36]:
np.sum(waveform)

0.29558018025978611

In [37]:
np.sum(wave_form)

-0.38118343556410023